## Deep Learning for Time Series 

Deep Learning is a subset of machine learning and excels when dealing with large and complex data, as it can extract complex features with minimal human involvement. Deep learning works well with structured and unstructured data and can be used in supervised, unsupervised and semi-supervised learning. 

This chapter focuses on using deep learning for time series forecasting - using different deep learning architectures suitable for sequential data such as time series data. There are different deep learning architectures for solving various problems: 

* Recurrent Neural Networks (RNNs)
* Long-Short Term Memory (LSTM)
* Gated Recurrent Unit (GRU)
* Convolutional Neural Networks (CNNs)
* Autoencoders
* Generative Adversarial Networks (GANs)

In [4]:
class Standardize:
    def __init__(self, df, split=0.10):
        self.data = df 
        self.split = split 

    def split_data(self):
        n = int(len(self.data) * self.split)
        train, test = self.data.iloc[:-n], self.data.iloc[-n:]
        n = int(len(train) * self.split)
        train, val = train.iloc[:-n], train.iloc[-n:]
        assert len(test) + len(train) + len(val) == len(self.data)
        return train, test, val 
    
    def _transform(self, data):
        data_s = (data - self.mu)/self.sigma 
        return data_s 
    
    def fit_transform(self):
        train, test, val = self.split_data()
        self.mu, self.sigma = train.mean(), train.std()
        train_s = self._transform(train)
        test_s = self._transform(test)
        val_s = self._transform(val)
        return train_s, test_s, val_s 
    
    def inverse(self, data):
        return (data * self.sigma)+self.mu 
    
    def inverse_y(self, data):
        return (data * self.sigma[-1])+self.mu[-1]
    


In [7]:
def one_step_forecast(df, window):
    d = df.values 
    x = []
    n = len(df)
    idx = df.index[:-window]
    for start in range(n-window):
        end = start + window
        x.append(d[start:end])
    cols = [f'x_{i}' for i in range(1, window+1)]
    x = np.array(x).reshape(n-window, -1)
    y = df.iloc[window:].values
    df_xs = pd.DataFrame(x, columns=cols, index=idx)
    df_y = pd.DataFrame(y.reshape(-1), columns=['y'], index=idx)

    return pd.concat([df_y, df_xs], axis=1).dropna()

In [8]:
# Example 

import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
from pathlib import Path 
import warnings
warnings.filterwarnings('ignore')
path = Path('../TimeSeriesAnalysisWithPythonCookbook/Data/')
energy = pd.read_csv(path.joinpath('energy_consumption.csv'), index_col='Month', parse_dates=True)
energy.columns = ['y']
energy.index.freq = 'MS'
en_df = one_step_forecast(energy, 10)

In [9]:
# Scale the data using the updated Standardize class 
scale_en = Standardize(en_df)

In [10]:
train_en, test_en, val_en = scale_en.fit_transform()

Deep Learning libraries can be broken down into either **low-level, high-level** or both. High-level libraries allow for quick prototyping and experimentation when testing various architectures, such as the case with Keras. A low-level library gives us more flexibility and control, but we will have to define more aspects of a model's architecture - PyTorch and Tensorflow are examples of low-level libraries. 

## Forecasting with an RNN using Keras 

RNNs initially entered the spotlight with NLP, as they were designed for sequential data, where past observations, such as words, have a strong influence on determining the next word in a sentence. This need for the artificial neural network to retain memory (hidden state) inspired the RNN architecture. Similarly, time series data is also sequential, and since past observations influece future observations, it also needs a network with memory. 

In RNNs, there is a feedback loop where the output of one node or neuron is fed back (the recursive part) as input, allowing the network to learn from a prior time step acting as a memory. 

In an RNN we have two outputs and two sets of weights: $W_{X}$ for the input and $W_{H}$ for the hidden state or memory



In [16]:
import tensorflow as tf 

import tf.keras.Sequential 

# import Sequential 
#from tensorflow import keras 
# from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsoluteError 
# from keras.layers import Dense, SimpleRnn, Dropout

ModuleNotFoundError: No module named 'tf'